In [1]:
from coincurve import PrivateKey

def bech32_encode(hrp: str, data: bytes) -> str:
    charset = "qpzry9x8gf2tvdw0s3jn54khce6mua7l"
    def convertbits(data: bytes, frombits: int, tobits: int, pad: bool = True) -> list[int]:
        acc = 0
        bits = 0
        ret = []
        maxv = (1 << tobits) - 1
        for b in data:
            acc = (acc << frombits) | b
            bits += frombits
            while bits >= tobits:
                bits -= tobits
                ret.append((acc >> bits) & maxv)
        if pad and bits:
            ret.append((acc << (tobits - bits)) & maxv)
        elif bits >= frombits or ((acc << (tobits - bits)) & maxv):
            raise ValueError("Invalid bits conversion")
        return ret

    def bech32_polymod(values: list[int]) -> int:
        GENERATOR = [0x3b6a57b2, 0x26508e6d, 0x1ea119fa, 0x3d4233dd, 0x2a1462b3]
        chk = 1
        for v in values:
            b = (chk >> 25) & 0xFF
            chk = ((chk & 0x1FFFFFF) << 5) ^ v
            for i in range(5):
                if (b >> i) & 1:
                    chk ^= GENERATOR[i]
        return chk

    def bech32_hrp_expand(hrp: str) -> list[int]:
        return [ord(x) >> 5 for x in hrp] + [0] + [ord(x) & 31 for x in hrp]

    def bech32_create_checksum(hrp: str, data: list[int]) -> list[int]:
        values = bech32_hrp_expand(hrp) + data
        polymod = bech32_polymod(values + [0, 0, 0, 0, 0, 0]) ^ 1
        return [(polymod >> 5 * (5 - i)) & 31 for i in range(6)]

    data5 = convertbits(data, 8, 5)
    checksum = bech32_create_checksum(hrp, data5)
    combined = data5 + checksum
    return hrp + "1" + "".join([charset[d] for d in combined])

def generate_nsec() -> str:
    privkey = PrivateKey()
    nsec_bytes = privkey.secret
    nsec = bech32_encode("nsec", nsec_bytes)
    return nsec


In [14]:
import asyncio
import sys
from datetime import datetime
from typing import Any

# For secp256k1 key operations
try:
    from coincurve import PrivateKey
except ImportError:
    print("Please install coincurve: pip install coincurve")
    sys.exit(1)

from sixty_nuts.relay import NostrRelay, NostrFilter


# Common Nostr relays
DEFAULT_RELAYS = [
    "wss://relay.damus.io",
    "wss://nos.lol",
    "wss://relay.nostr.band",
    "wss://relay.snort.social",
]

# Common event kinds for reference
EVENT_KINDS = {
    0: "Profile metadata",
    1: "Text note",
    2: "Recommend relay",
    3: "Contact list",
    4: "Encrypted direct message",
    5: "Event deletion",
    6: "Repost",
    7: "Reaction",
    40: "Channel creation",
    41: "Channel metadata",
    42: "Channel message",
    43: "Channel hide message",
    44: "Channel mute user",
    1984: "Reporting",
    9734: "Zap request",
    9735: "Zap",
    10000: "Mute list",
    10001: "Pin list",
    10002: "Relay list metadata",
    10019: "Relay recommendations",
    17375: "NIP-60 wallet metadata",
    7374: "NIP-60 quote",
    7375: "NIP-60 token",
    7376: "NIP-60 history",
}


def bech32_decode(bech: str) -> tuple[str, bytes]:
    """Decode bech32 string and return hrp and data."""
    CHARSET = "qpzry9x8gf2tvdw0s3jn54khce6mua7l"

    if not bech:
        raise ValueError("Empty bech32 string")

    bech = bech.lower()
    pos = bech.rfind("1")
    if pos < 1 or pos + 7 > len(bech) or pos + 1 + 6 > len(bech):
        raise ValueError("Invalid bech32 format")

    hrp = bech[:pos]
    data = bech[pos + 1 :]

    # Convert data part to integers
    decoded = []
    for char in data:
        if char not in CHARSET:
            raise ValueError(f"Invalid character in bech32: {char}")
        decoded.append(CHARSET.index(char))

    # Verify checksum (simplified - not implementing full bech32 verification)
    if len(decoded) < 6:
        raise ValueError("Invalid bech32 data length")

    # Convert from 5-bit to 8-bit
    data_part = decoded[:-6]  # Remove checksum

    # Convert 5-bit groups to bytes
    bits = 0
    value = 0
    result = []

    for group in data_part:
        value = (value << 5) | group
        bits += 5
        if bits >= 8:
            result.append((value >> (bits - 8)) & 255)
            bits -= 8

    if bits >= 5:
        raise ValueError("Invalid padding in bech32")

    return hrp, bytes(result)


def nsec_to_pubkey(nsec: str) -> str:
    """Convert nsec to hex pubkey."""
    try:
        hrp, data = bech32_decode(nsec)

        if hrp != "nsec":
            raise ValueError("Not a valid nsec (expected 'nsec' prefix)")

        if len(data) != 32:
            raise ValueError("Invalid private key length")

        # Create coincurve private key and derive public key
        private_key = PrivateKey(data)
        print(private_key.to_hex(), "pr")
        public_key_bytes = private_key.public_key.format(compressed=True)

        # Return hex pubkey (skip the first byte which is the compression flag)
        return public_key_bytes[1:].hex()

    except Exception as e:
        raise ValueError(f"Invalid nsec: {e}")


def format_timestamp(timestamp: int) -> str:
    """Format Unix timestamp to readable string."""
    return datetime.fromtimestamp(timestamp).strftime("%Y-%m-%d %H:%M:%S")


def get_kind_description(kind: int) -> str:
    """Get human-readable description for event kind."""
    return EVENT_KINDS.get(kind, f"Unknown kind {kind}")


async def fetch_events_from_relay(
    relay_url: str, pubkey: str, kinds: list[int] | None = None, limit: int = 50
) -> list[Any]:
    """Fetch events from a single relay."""
    relay = NostrRelay(relay_url)

    try:
        print(f"Connecting to {relay_url}...")

        # Build filter
        filter_dict: NostrFilter = {
            "authors": [pubkey],
            "limit": limit,
        }

        # Add kinds filter if specified
        if kinds is not None:
            filter_dict["kinds"] = kinds

        filters: list[NostrFilter] = [filter_dict]

        events = await relay.fetch_events(filters, timeout=10.0)
        print(f"  Found {len(events)} events from {relay_url}")
        return events

    except Exception as e:
        print(f"  Error fetching from {relay_url}: {e}")
        return []
    finally:
        await relay.disconnect()


async def fetch_all_events(
    nsec: str,
    relays: list[str] | None = None,
    kinds: list[int] | None = None,
    limit: int = 50,
) -> None:
    """Fetch all events from the given nsec across multiple relays."""

    # Convert nsec to pubkey
    try:
        pubkey = nsec_to_pubkey(nsec)
        print(f"Fetching events for pubkey: {pubkey}")
    except ValueError as e:
        print(f"Error: {e}")
        return

    # Use default relays if none provided
    if relays is None:
        relays = DEFAULT_RELAYS

    kinds_str = f" (kinds: {kinds})" if kinds else " (all kinds)"
    print(f"Searching {len(relays)} relays{kinds_str}...")

    # Fetch from all relays in parallel
    tasks = [fetch_events_from_relay(relay, pubkey, kinds, limit) for relay in relays]
    results = await asyncio.gather(*tasks, return_exceptions=True)

    # Collect all events and deduplicate by ID
    all_events = {}
    for result in results:
        if isinstance(result, list):
            for event in result:
                all_events[event["id"]] = event

    # Sort by creation time (newest first)
    events = sorted(all_events.values(), key=lambda x: x["created_at"], reverse=True)

    print(f"\n=== Found {len(events)} unique events ===\n")

    # Display the events
    for i, event in enumerate(events, 1):
        kind_desc = get_kind_description(event["kind"])
        print(f"[{i}] {format_timestamp(event['created_at'])}")
        print(f"Kind: {event['kind']} ({kind_desc})")
        print(f"ID: {event['id']}")
        print(
            f"Content: {event['content']}"
        )

        # Show tags if any
        if event["tags"]:
            print(f"Tags: {len(event['tags'])} tag(s)")
            for tag in event["tags"][:3]:  # Show first 3 tags
                print(f"  {tag}")
            if len(event["tags"]) > 3:
                print(f"  ... and {len(event['tags']) - 3} more")

        print("-" * 80)


def parse_kinds(kinds_str: str) -> list[int]:
    """Parse comma-separated kinds string into list of integers."""
    try:
        return [int(k.strip()) for k in kinds_str.split(",") if k.strip()]
    except ValueError as e:
        raise ValueError(f"Invalid kind number: {e}")


async def main(nsec: str):
    # Parse arguments
    kinds = None
    custom_relays = []

    for arg in sys.argv[2:]:
        if arg.startswith("--kinds="):
            kinds_str = arg[8:]  # Remove "--kinds="
            try:
                kinds = parse_kinds(kinds_str)
                print(f"Filtering for kinds: {kinds}")
            except ValueError as e:
                print(f"Error parsing kinds: {e}")
                return
        elif arg.startswith("wss://") or arg.startswith("ws://"):
            custom_relays.append(arg)
        else:
            print(f"Unknown argument: {arg}")
            return

    relay_list = custom_relays if custom_relays else None
    await fetch_all_events(nsec, relay_list, kinds)



In [3]:
nsec2 = generate_nsec()
"nsec1g076as5d528uenjkx7xcwwjv86569ax5hkd3vl5a8e00hk6zhcjsn8pwqe", nsec2

('nsec1g076as5d528uenjkx7xcwwjv86569ax5hkd3vl5a8e00hk6zhcjsn8pwqe',
 'nsec12t92hp4zcmmcngfk2xxlh2pvq56f6pz89s392zyvylg7je5chrgsamrq3v')

In [21]:
await main("nsec1g076as5d528uenjkx7xcwwjv86569ax5hkd3vl5a8e00hk6zhcjsn8pwqe")


43fdaec28da28fccce56378d873a4c3ea9a2f4d4bd9b167e9d3e5efbdb42be25 pr
Fetching events for pubkey: 3976c2fefbb924e7d0372cf8064c1855bc2fbd89369d30dc1dc54e7b0e363eac
Searching 4 relays (all kinds)...
Connecting to wss://relay.damus.io...
Connecting to wss://nos.lol...
Connecting to wss://relay.nostr.band...
Connecting to wss://relay.snort.social...
  Found 3 events from wss://nos.lol
  Found 3 events from wss://relay.damus.io
  Found 2 events from wss://relay.snort.social
  Found 3 events from wss://relay.nostr.band

=== Found 3 unique events ===

[1] 2025-06-04 15:41:47
Kind: 7375 (NIP-60 token)
ID: 9924667c613fb404b1daa192e8cc374f4c610dd173d697ab28b7d903e7c1c239
Content: AhNKOe0d6wT2Qlrpd+dfU6zy8zGql9pX6kS+kaqJ3V4NKs3gNCWYSleorkxplVCHFFvGBwghjSNeHHcdwrWvcVBQsYSAyemcDBwEgFaX0CRhfMtTmeJDZYkYUng5Czq/KMr7+TxY9zW/qKR8tYqNZx5T82c4Yb80OXyR2wQ2saTvMYZ4fh6qoBGfusEtMO/qgZNYrwBX5H47mi7JEOcWXegCfZx4VGJtxbn4OjEzqL7nd5jF0JvaCNplBqgkC86V67TZ+Wy3LVNsbrhMM7b/dyJCuh7wuirM6CeFW/mCp9/6zRk4ttvN6GmWTmUNsWzMb0P

In [16]:
from sixty_nuts.crypto import NIP44Encrypt
from coincurve import PrivateKey

sender_nsec = "nsec1g076as5d528uenjkx7xcwwjv86569ax5hkd3vl5a8e00hk6zhcjsn8pwqe"
hrp, sk_bytes = bech32_decode(sender_nsec)
sender_priv = PrivateKey(sk_bytes)
sender_pub_compressed = sender_priv.public_key.format(compressed=True).hex()

msg = "Ai917eoTypQZRbFLzNSGVppQ4InuSNlcj4CQd/bkzgR2yM1M28IC5tHKMzw7WpHW5S/oKIGP7xC39IJ28WlM1b7FcV33jEPtFBS5k0AvxHhbdle9HjxBz9S923wXBVGeD53LklUQGONL/NPL2UaduyOBfTDaFJRURJ31DVwny20H0mxPJOqnom/wSTgOUtqtZKoyyjDguiKdNM/ycz5CuT4YWmKejjsfPanAYSX3+CmWSRSNQNxBZmGQYVduOVOc4OLjHaJnxc94deIX9vMCULwrkfFN6tb0799rkc7XnHqBP5+FBUUNBehvHS35E6YDF1+tT7gCFVc7Z1CbYs/TVm7pGpvHDgVv89ITgC3est0TvL7nB63l0mgOoLV4VbJCa4HxSgr42dUhxD4amdYpLHil9cT2fynzLYwR4TbMltrAmnqBCr6+ZC3P4prrgislwdF1jfJSWeXn+xDAg+ueq+P4qj65g7VOugybbBnVZ1pL1GLObDfLvqcs5H12LWpH0emfFnPirDWSxJwQzt/SsnvPp0Ayk6O/BFUUbAqv/UYe6sDs62eq4t7U6kGRQfWOjCJ10ypAyIKOuz7p788IlPK3dJcTGd7HlJdgzdv2b4LRRLTtTw49KQ86vPcyabOqiKZ9rWShZdSPsOSiZHZxyGGsKbhAO16ug6Ybi6GFn279L2yGJbbL3OcIM/US/U7fs/lKaSikIB48FKB0xh11XXY78gPm2EQwjZAzLFCS/qe6Z2DfjijslvhoD5+L32GuU0FnSDv6QaGYdhQ1vXs3sjRgsTdhxSgby1Zm2ausNw5co85dldGonRiSnqfagR2hszQSCYHAQBJZz4m1xpK429pg0V4th//pi+zOKv47j6YlTgufanvzXPU/oZ6cAwpWcmz6J/F7ywnXJrZeM7Zri5sDcVetUXQqLEvZewywh7pQsgd+G2k5QH8ij9I1K8f8bq+vJrGutxBeMeHEQrRqN6J8Lth8TLVCYJ2VJ1aGe7r/l4E98B72mujpSJ/mT6WE3NzdZYu0DbT5nhoNgY5AY70u8OIP/qAaihq05E42YcuKAvZ1N3RfDddY6fUklBbmb1JF83UnvkGbv9oatSUpcli64EACF3tYOnV+iqL0i+FccxoSn5UqZOFmtOu3J5KpggZpbP6phvahMT8VfEnqtvSZYyrg3nz4YfVHrcNwnlV4uPs5FL7hdiRQtN447Ice8dXVctZOB6ZrOyRwO8dk4MOVyb7mUkcc8+oVRYDHl1ckLKTduu4mHoqyyZooWDLVFpvO"
msg = NIP44Encrypt.decrypt(msg, sender_priv, sender_pub_compressed)
print(msg)
NIP44Encrypt.encrypt(msg, sender_priv, sender_pub_compressed)


{"mint":"https://mint.minibits.cash/Bitcoin","proofs":[{"secret":"9c52c2a28703f707ae148184a505f044ffec17872f3343dab34504fceebdbd3b","C":"02fa3460e7dda04f0848b6e66e87ef369b77a06f2304b9e7f0bd0cd78f98aaa7f2","amount":8,"id":"00500550f0494146","dleq":{"r":"0e13dc04b571059b9ad775d898d4c145d87c4aecff326efed8b73fce43b0badd","s":"a1a64c7f82bbc111c7ff862be9375fecbe89d8066ecb3c0acfb911b42cf65e11","e":"64a10585e9de5b2044cee5518332c5267074f7adc3c4d51ed45164a326739be3"}},{"secret":"4e329ba084c0b63e0319698fff1451cf6c836e9c283a89e0c90f6c19ecb0ab4d","C":"033c4701cae515b8399537bad0cd63ac60c9bfbec7698c892f6d246b7ca1a4523d","amount":4,"id":"00500550f0494146","dleq":{"r":"6f9558c0346565628b43cf7836333e91018820a1d0a1edd328d02e6d385c5ec5","s":"468ab7af450b9f0cbb3f72e1aa2eb51aeb6723c8e30096d64ce47afcda151f1c","e":"75e25b28dbd408fa19abb1a9babe35fb969052f7e73214a7549cfce35957b8e9"}}]}


'AnCHvTFU8cRdhVEKQ3pLqlOpa7rVO/hxvU1RhbzOIRPr7ifv9Osh1eARW7Vj1R0iW3s57iiwRM2bjEaDPeeRZok6oK7YqThZ8hhlPQ0d7tvz30L2Asg0UVuC1n4HG9PSAqLDKmEWviSrmV0E6yJS1A9FQs/PCnMoh7otT48CrQc4rInjstF1Oi3HuRgJx07Ys0fuPy9mpXJMtzvJGZMOfSzHXqqEEFaZkYLFiQ3WoueY0aiiDXJnmDGBAIGNtb1wzwGQZiDvkwDZawZX3WIXh0QQPRcMbFDTJjK7S4sAYUO4/GHJuULb6f229/WzyW6sDgKKxLoqKFx5G/O1Xjya+E7kREA+qqYElLGuiKp26g3bxefT2+f0G/w7TvB+O0Kp7Os95aY5hN71IFf/oiDNUaaHNnPkVCVe7eSpo1if5qpGgWbdOrLncVl6kyBGr7y1CW6QAlxcUqGPV4v87/zDlum6FqAt9fwxTMbfAELeeaObFMseLhE9n5oselgMHFgUTsKaVdUWzsV704PiPAIRNpcxDmrQ7DJFakbtHhXU3CBozcXnAAbGYp4zAJ9dbgQHoJFqfCUyklgJb072IF+G4XJVg6pj16LAH0U9cLvVJyGqVL61qGWZCpKIxf3/3utEO/c/tyqJlgMGOkr3kLHFwlWuVFRcP6zHeNaS3jnXZpYFcIR2F49vOiHal7bTtnV5KTy0aaGtT7mk1+vdRPLK4OoZHxUJO+UokjPD3i4i4NBqhQn2B7j9HC2JCEAS2YrWLuUR6tUfyy/+s0ihMQuTmfVIa2TJcE1ejeQeY4kx7pchqi/pr6sLqkmYMeQNw8Tafh8IUB48lgz8qCsCtwqWGLjIbZElHRy6F1MBo/OyPrE09ezYTgk9EBhnsahZJe45btNurVn7KwrSyWHK2YSy/Cm0ktxNlajk1BRF1Jk5FaBgK96qV1luW9SGM+mMptQjXbKhBBgvaBmtx3P8p+0mUv+iWfJj+twtK+rqFbN

In [25]:
from sixty_nuts.wallet import Wallet

wallet = await Wallet.create("nsec1g076as5d528uenjkx7xcwwjv86569ax5hkd3vl5a8e00hk6zhcjsn8pwqe")

await wallet.send(20)

ConnectError: All connection attempts failed